In [1]:
import itertools
import sys, os

import numpy as np
import pandas as pd
from scipy.special import comb
from scipy import stats
import scipy.cluster.hierarchy as hac
import matplotlib.pyplot as plt
plt.style.use('classic')
%matplotlib inline
import numpy as np
import pandas as pd
import seaborn as sns

from collections import Counter
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

sns.set(rc={'figure.figsize':(15,8)})
sns.set_context('poster')

In [2]:
%load_ext autoreload
%autoreload 2
#import CCPA_lib as cp
import sorf_lib as sf

In [3]:
sns.set_context('talk')
sns.set_style('white')


![](http://oregonstate.edu/instruct/bb450/fall14/stryer7/2/table_02_02.jpg)

https://www.uniprot.org/uniprot/Q7V735

http://tigrfams.jcvi.org/cgi-bin/HmmReportPage.cgi?acc=TIGR03798

https://www.ebi.ac.uk/training/online/course/interpro-functional-and-structural-analysis-protei/sequence-searching/searching-interpro-batc

http://www.ebi.ac.uk/interpro/sequencesearch/iprscan5-S20190707-131508-0462-76111813-p1m

https://www.ebi.ac.uk/Tools/services/rest/iprscan5/result/iprscan5-S20190707-131508-0462-76111813-p1m/json

In [4]:
oorder = ['Intergenic', 'Antisense', 'Out of Frame','In Frame',  'Annotated', ]
opalette = sns.color_palette("rainbow", 5).as_hex()
#del opalette[4]
genome='MIT9313'

In [5]:
found_sorf_df = pd.read_pickle('found_sorf_ISME16.pkl.gz')

In [6]:
found_sorf_df.drop_duplicates(subset='orfid', inplace=True)

# Gene Families

In [7]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    display(found_sorf_df['product'].value_counts())

hypothetical protein                                                                            66
Nif11-like leader peptide family natural product precursor                                      18
                                                                                                11
high light inducible protein                                                                     4
DUF4278 domain-containing protein                                                                2
2Fe-2S iron-sulfur cluster binding domain-containing protein                                     2
30S ribosomal protein S13                                                                        1
pterin-4-alpha-carbinolamine dehydratase                                                         1
DUF2973 domain-containing protein                                                                1
DUF3181 family protein                                                                           1
cupin doma

In [8]:
found_sorf_df['family2'] = found_sorf_df['family']
found_sorf_df.loc[found_sorf_df['product'].str.contains('ribosomal'), 'family2'] = 'ribosomal'
found_sorf_df.loc[found_sorf_df['product'].str.contains('photosystem'), 'family2'] = 'photosystem'
found_sorf_df.loc[found_sorf_df['product'].str.contains('hypothetical'), 'family2'] = 'hypothetical'
found_sorf_df.loc[found_sorf_df['product'] == '', 'family2'] = 'N/A'
found_sorf_df.loc[found_sorf_df['product'].str.contains('domain-containing protein') &
                  found_sorf_df['product'].str.startswith('DUF') , 'family2'] = 'DUF'
found_sorf_df.loc[found_sorf_df['product'].str.contains('family protein') &
                  found_sorf_df['product'].str.startswith('DUF') , 'family2'] = 'DUF'
found_sorf_df.loc[found_sorf_df['product'].str.contains('high light inducible protein'), 'family2'] = 'HLIP'
found_sorf_df.loc[found_sorf_df['product'].str.contains('carboxysome'), 'family2'] = 'carboxysome'
found_sorf_df.loc[found_sorf_df['family2'].isna(), 'family2'] = 'other'
found_sorf_df.loc[~found_sorf_df['family'].isna(), 'family2'] = found_sorf_df.loc[~found_sorf_df['family'].isna(), 'family']


In [9]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None, 'display.max_colwidth', 100000):  # more options can be specified also
    #display(found_sorf_df.loc[found_sorf_df.family2.isna(), 'product'].value_counts())    
    display(found_sorf_df.loc[found_sorf_df.family2.isna(), 'product'].sort_values().reset_index(drop=True))

Series([], Name: product, dtype: object)

In [10]:
found_sorf_df.groupby(['family2'])['product'].value_counts()

family2    product                          
CCRG-1     guanylate-binding protein            1
CCRG-2     CCRG-2 family RiPP                   1
           hypothetical protein                 1
DUF        DUF4278 domain-containing protein    2
           DUF1816 domain-containing protein    1
                                               ..
ribosomal  50S ribosomal protein L28            1
           50S ribosomal protein L29            1
           50S ribosomal protein L31            1
           50S ribosomal protein L32            1
           50S ribosomal protein L33            1
Name: product, Length: 73, dtype: int64

In [11]:
found_sorf_df.loc[found_sorf_df['family2'] == 'other']['product'].value_counts()

2Fe-2S iron-sulfur cluster binding domain-containing protein                                    2
NAD(P)H-quinone oxidoreductase subunit O                                                        1
NifU family protein                                                                             1
ssl1498 family light-harvesting-like protein                                                    1
NAD(P)-dependent oxidoreductase                                                                 1
BolA family transcriptional regulator                                                           1
transcriptional repressor                                                                       1
chemotaxis protein                                                                              1
CopG family transcriptional regulator                                                           1
YlxR family protein                                                                             1
glutaredoxin 3      

# BLAST vs SORFS

In [12]:
def is_in_found_sorf(r):
    orfid = r.id.replace('lcl|','')
    return found_sorf_df.orfid.isin([orfid]).sum() > 0
    

In [13]:
accession = sf.get_accession(genome)
orf_dpath = 'orffinder_tmp'
orf_fpath = os.path.join(orf_dpath, f'{accession}.orffinder.fa')

In [14]:
from Bio import SeqIO
records = (r for r in SeqIO.parse(orf_fpath, "fasta") if is_in_found_sorf(r))

In [15]:
out_fpath = os.path.join(orf_dpath, f'{genome}_found_proteome.fasta')
count = SeqIO.write(records, out_fpath, "fasta")
print("Saved %i records from %s to %s" % (count, orf_fpath, out_fpath))

Saved 169 records from orffinder_tmp\BX548175.orffinder.fa to orffinder_tmp\MIT9313_found_proteome.fasta


 mit9313_found_vs_pro_syn    https://blast.ncbi.nlm.nih.gov/Blast.cgi?CMD=Get&RID=8P2ACMPV014    
 mit9313_found_vs_nr         https://blast.ncbi.nlm.nih.gov/Blast.cgi?CMD=Get&RID=8P26YCM7014
 mit9313_found_vs_swisprot   https://blast.ncbi.nlm.nih.gov/Blast.cgi?CMD=Get&RID=8SHSH1FU014
 

In [16]:
blast_dpath = os.path.join('orffinder_tmp','blast results')
blast_vs_nr_dpath = os.path.join(blast_dpath, 'mit9313_found_vs_nr-Alignment.xml')
blast_vs_pro_dpath = os.path.join(blast_dpath, 'mit9313_found_vs_pro_syn-Alignment.xml')
blastn_vs_pro_dpath = os.path.join(blast_dpath, 'mit9313_blastn_vs_pro_syn-Alignment.xml')

blast_vs_sorf_dpath = os.path.join(blast_dpath, 'mit9313_sorf_all_vs_all.xml')
blast_vs_swisprot_dpath = os.path.join(blast_dpath, 'mit9313_found_vs_swisprot-Alignment.xml')

os.listdir(blast_dpath)

['BHXV0U5D014-Alignment.xml',
 'mit9313_blastn_vs_pro_syn-Alignment.xml',
 'mit9313_found_vs_pro_syn-Alignment.xml',
 'mit9313_found_vs_swisprot-Alignment.xml',
 'mit9313_sorf_all_vs_all.xml',
 'old']

In [17]:
from Bio.Blast import NCBIXML


# BLAST vs SORF

In [18]:
def blast_rec_to_dict(blast_rec):
    query_id = blast_rec.query.replace(' unnamed protein product','').replace('lcl|','')
    results = [{
        'query' : query_id,
        'aligment_id' : alignment.hit_def.split()[0].replace('lcl|', ''), 
        'alignment_def' : alignment.hit_def,
        'coverage': np.round(np.mean([hsp.align_length / blast_rec.query_length for hsp in alignment.hsps]), decimals=2),
        'identity': np.round(np.mean([hsp.identities / blast_rec.query_length for hsp in alignment.hsps]), decimals=2),
        
        }
        for alignment in blast_rec.alignments if alignment.hit_id != query_id
    ]
    return results

In [19]:
fh = open(blast_vs_sorf_dpath)
blast_records = NCBIXML.parse(fh)
reslist = [blast_rec_to_dict(b) for b in blast_records]
blast_vs_sorf_df = pd.DataFrame([i for lst in reslist for i in lst])

In [20]:
fh.close()

In [21]:
for query in blast_records:
    for alignment in query.alignments:
        for hsp in alignment.hsps:
            print('coverage', hsp.align_length / query.query_length)
            print('identitiy', hsp.identities/ hsp.align_length)

In [22]:
blast_vs_sorf_df.head()

,query,aligment_id,alignment_def,coverage,identity
0,ORF533_BX548175.1:237054:237281,ORF533_BX548175.1:237054:237281,lcl|ORF533_BX548175.1:237054:237281 unnamed pr...,1.00,1.00
1,ORF2049_BX548175.1:913137:913382,ORF2049_BX548175.1:913137:913382,lcl|ORF2049_BX548175.1:913137:913382 unnamed p...,1.00,1.00
2,ORF2049_BX548175.1:913137:913382,ORF26372_BX548175.1:2236618:2236379,lcl|ORF26372_BX548175.1:2236618:2236379 unname...,0.93,0.75
3,ORF2049_BX548175.1:913137:913382,ORF26370_BX548175.1:2237020:2236769,lcl|ORF26370_BX548175.1:2237020:2236769 unname...,0.79,0.72
4,ORF2049_BX548175.1:913137:913382,ORF20045_BX548175.1:274995:274750,lcl|ORF20045_BX548175.1:274995:274750 unnamed ...,0.84,0.69


In [23]:
def create_sorfs_cluster(blast_vs_sorf_df, cluster_sorfs):
    new_alignments = set(
        blast_vs_sorf_df.loc[blast_vs_sorf_df['query'].isin(cluster_sorfs), 'aligment_id'].unique()
    )
    new_queries = set(
        blast_vs_sorf_df.loc[blast_vs_sorf_df['aligment_id'].isin(cluster_sorfs), 'query'].unique()
    )
    if (new_alignments <= cluster_sorfs) and (new_queries <= cluster_sorfs):
        return cluster_sorfs
    else:
        new_cluster_sorfs = cluster_sorfs | new_alignments | new_queries
        return create_sorfs_cluster(blast_vs_sorf_df, new_cluster_sorfs)
    
    

In [24]:
def create_sorfs_cluster_sel_root(blast_vs_sorf_df, visited):
    not_visited = blast_vs_sorf_df.loc[~blast_vs_sorf_df['query'].isin(visited), 'query'].unique()
    if len(not_visited):
        return create_sorfs_cluster(blast_vs_sorf_df, {not_visited[0]})
    #else
    not_visited2 = blast_vs_sorf_df.loc[~blast_vs_sorf_df['aligment_id'].isin(visited), 'aligment_id'].unique()
    if len(not_visited2):
        return create_sorfs_cluster(blast_vs_sorf_df, {not_visited2[0]})
    # else
    return None
    


In [25]:
visited = set()
clusters = []
while(True):
    new_cluster = create_sorfs_cluster_sel_root(blast_vs_sorf_df, visited)
    if new_cluster is None:
        break
    visited = visited | new_cluster
    clusters.append(new_cluster)
    

In [26]:
len(clusters)

139

In [27]:
clusters

[{'ORF533_BX548175.1:237054:237281'},
 {'ORF15620_BX548175.1:2243295:2243047',
  'ORF15637_BX548175.1:2235903:2235652',
  'ORF15638_BX548175.1:2235486:2235229',
  'ORF15639_BX548175.1:2235150:2234896',
  'ORF20045_BX548175.1:274995:274750',
  'ORF20053_BX548175.1:273630:273355',
  'ORF2049_BX548175.1:913137:913382',
  'ORF20960_BX548175.1:2244944:2244663',
  'ORF20982_BX548175.1:2236277:2236026',
  'ORF25351_BX548175.1:278051:277779',
  'ORF25353_BX548175.1:276446:276159',
  'ORF25368_BX548175.1:272714:272418',
  'ORF25455_BX548175.1:233492:233262',
  'ORF26352_BX548175.1:2244112:2243855',
  'ORF26353_BX548175.1:2243725:2243456',
  'ORF26370_BX548175.1:2237020:2236769',
  'ORF26372_BX548175.1:2236618:2236379',
  'ORF5422_BX548175.1:182116:182298',
  'ORF7032_BX548175.1:905452:905715'},
 {'ORF2068_BX548175.1:922530:922661'},
 {'ORF2106_BX548175.1:940263:940514'},
 {'ORF2222_BX548175.1:986481:986612'},
 {'ORF18490_BX548175.1:968157:967891', 'ORF2321_BX548175.1:1030986:1031147'},
 {'ORF23

In [28]:
real_clusters = [i for i in clusters if len(i)>1]
for i,c in enumerate(real_clusters, start=1):
    found_sorf_df.loc[found_sorf_df.orfid.isin(c), 'cluster_id'] = i

In [29]:
len(real_clusters)

11

In [30]:
found_sorf_df.head()

,orfid,contig,left,right,start,stop,strand,aaseq,len_nn,len_aa,...,AAs,MW [kDa],calc. pI,tempaccession,max_xCorr,multiple unique peptides,has unique peptides,ms_detected,family2,cluster_id
578,ORF533_BX548175.1:237054:237281,BX548175.1,237054,237281,237054,237281,+,MLYLVAPAGDGRDVYATLYAQRMFFLVTLQPRGAQFEVIPYQDARH...,228,75,...,75,8.89242,6.771,unnamed protein product - [ORF533_BX548175.1:2...,4.305928,True,True,True,other,NaN
2285,ORF2049_BX548175.1:913137:913382,BX548175.1,913137,913382,913137,913382,+,MSEEQLKAFIAKVQGDTSLQEQLKAEGADVVAIAKAAGFSITTEDL...,246,81,...,81,8.50516,4.48779,unnamed protein product - [ORF2049_BX548175.1:...,5.280437,True,True,True,ProcA,1.0
2316,ORF2068_BX548175.1:922530:922661,BX548175.1,922530,922661,922530,922661,+,MMGKTKQHKKLVSWAEKAELCATREEAQKLIRKAEKAHAKLES,132,43,...,43,4.93366,9.71533,unnamed protein product - [ORF2068_BX548175.1:...,3.770810,False,True,True,N/A,NaN
2380,ORF2106_BX548175.1:940263:940514,BX548175.1,940263,940514,940263,940514,+,MGFLHELEISDGGHRFELASHDHPDHHHLVCVRCGRTEEFESKPVL...,252,83,...,83,9.29551,6.50732,unnamed protein product - [ORF2106_BX548175.1:...,3.996620,True,True,True,other,NaN
2516,ORF2222_BX548175.1:986481:986612,BX548175.1,986481,986612,986481,986612,+,MAETATIRSDPDSSRNNRGKFVVAGYVLELAKIISDRRANQSS,132,43,...,43,4.72243,9.70068,unnamed protein product - [ORF2222_BX548175.1:...,4.124203,True,True,True,hypothetical,NaN


In [31]:
found_sorf_df.groupby(['cluster_id','family2'])['product'].value_counts()

cluster_id  family2       product                                                   
1.0         ProcA         Nif11-like leader peptide family natural product precursor    18
                          lantipeptide                                                   1
2.0         N/A                                                                          1
            hypothetical  hypothetical protein                                           1
3.0         HLIP          high light inducible protein                                   2
4.0         hypothetical  hypothetical protein                                           2
5.0         hypothetical  hypothetical protein                                           2
6.0         hypothetical  hypothetical protein                                           2
7.0         hypothetical  hypothetical protein                                           2
8.0         hypothetical  hypothetical protein                                           1
     

In [32]:
found_sorf_df.groupby(['cluster_id', 'otype', 'orfid', 'product', #'family'
                      ])['family'].unique().str.join(',').reset_index()

,cluster_id,otype,orfid,product,family
0,1.0,Annotated,ORF15620_BX548175.1:2243295:2243047,Nif11-like leader peptide family natural produ...,ProcA
1,1.0,Annotated,ORF15637_BX548175.1:2235903:2235652,Nif11-like leader peptide family natural produ...,ProcA
2,1.0,Annotated,ORF15638_BX548175.1:2235486:2235229,lantipeptide,ProcA
3,1.0,Annotated,ORF15639_BX548175.1:2235150:2234896,Nif11-like leader peptide family natural produ...,ProcA
4,1.0,Annotated,ORF20045_BX548175.1:274995:274750,Nif11-like leader peptide family natural produ...,ProcA
5,1.0,Annotated,ORF20053_BX548175.1:273630:273355,Nif11-like leader peptide family natural produ...,ProcA
6,1.0,Annotated,ORF2049_BX548175.1:913137:913382,Nif11-like leader peptide family natural produ...,ProcA
7,1.0,Annotated,ORF20960_BX548175.1:2244944:2244663,Nif11-like leader peptide family natural produ...,ProcA
8,1.0,Annotated,ORF20982_BX548175.1:2236277:2236026,Nif11-like leader peptide family natural produ...,ProcA
9,1.0,Annotated,ORF25353_BX548175.1:276446:276159,Nif11-like leader peptide family natural produ...,ProcA


In [33]:
found_sorf_df.loc[found_sorf_df.cluster_id.isna()]['otype'].value_counts()

Annotated       95
Out of Frame    17
Intergenic      10
In Frame         4
Antisense        2
Name: otype, dtype: int64

In [34]:
found_sorf_df.pivot_table(columns='otype', index='family2',aggfunc='count', values='orfid').sum()


otype
Annotated       127.0
Antisense         2.0
In Frame          7.0
Intergenic       11.0
Out of Frame     22.0
dtype: float64

In [35]:
found_sorf_df.otype.value_counts()

Annotated       127
Out of Frame     22
Intergenic       11
In Frame          7
Antisense         2
Name: otype, dtype: int64

# tBLASTn vs PRO /SYN genomes

In [36]:
def pro_blastn_to_description(blast_rec):
    query_id = blast_rec.query.replace(' unnamed protein product','').replace('lcl|','')
    results = [{
        'query' : query_id,
        'pro_syn' : a.hit_def,
        'pro_syn_hsps' : len(a.hsps),
        'pro_syn_coverage': np.round(np.mean([hsp.align_length / blast_rec.query_length for hsp in a.hsps]), decimals=2),
        'pro_syn_identity': np.round(np.mean([hsp.identities / blast_rec.query_length for hsp in a.hsps]), decimals=2),
        }
        for a in blast_rec.alignments
    ]
    return results

In [37]:
fh = open(blastn_vs_pro_dpath)
blast_records = NCBIXML.parse(fh)
reslist = [pro_blastn_to_description(b) for b in blast_records]
blastn_vs_pro_df = pd.DataFrame([i for lst in reslist for i in lst])

In [38]:
fh.close()

In [39]:
blastn_vs_pro_df.head()

,query,pro_syn,pro_syn_hsps,pro_syn_coverage,pro_syn_identity
0,ORF533_BX548175.1:237054:237281,"Prochlorococcus marinus str. MIT 9303, complet...",1,1.0,1.00
1,ORF533_BX548175.1:237054:237281,Prochlorococcus marinus MIT9313 complete genome,1,1.0,1.00
2,ORF533_BX548175.1:237054:237281,"Synechococcus sp. KORDI-49, complete genome",1,1.0,0.83
3,ORF533_BX548175.1:237054:237281,"Synechococcus sp. WH 8109, complete genome",1,1.0,0.83
4,ORF533_BX548175.1:237054:237281,"Synechococcus sp. KORDI-52, complete genome",1,1.0,0.81


In [40]:
grouped_blastn_vs_pro_df =  blastn_vs_pro_df.loc[
    ~blastn_vs_pro_df.pro_syn.str.contains('MIT9313')
    ].groupby(by='query').agg({'pro_syn': 'count', 'pro_syn_hsps': 'mean'}).reset_index()
grouped_blastn_vs_pro_df.rename(columns={
    'pro_syn': 'pro/syn genomes', 
    'pro_syn_hsps': 'mean pro/syn alignments', 
    'pro_syn_coverage': 'mean pro/syn coverage',
    'pro_syn_identity': 'mean pro/syn identity'},
                                inplace=True)

In [41]:
grouped_blastn_vs_9313_df =  blastn_vs_pro_df.loc[
    blastn_vs_pro_df.pro_syn.str.contains('MIT9313')
    ].groupby(by='query').agg({'pro_syn': 'count', 'pro_syn_hsps': 'mean'}).reset_index()
grouped_blastn_vs_9313_df.rename(columns={
    'pro_syn': 'mit9313', 
    'pro_syn_hsps': 'mit9313 alignments',
    'pro_syn_coverage': 'mean mit9313 coverage',
    'pro_syn_identity': 'mean mit9313 identity'},
                                inplace=True)

In [42]:
found_sorf_df = pd.merge(found_sorf_df, 
         grouped_blastn_vs_pro_df,
         left_on='orfid', right_on='query', how='left')

In [43]:
found_sorf_df = pd.merge(found_sorf_df, 
         grouped_blastn_vs_9313_df,
         left_on='orfid', right_on='query', how='left')

In [44]:
found_sorf_df.head()

,orfid,contig,left,right,start,stop,strand,aaseq,len_nn,len_aa,...,has unique peptides,ms_detected,family2,cluster_id,query_x,pro/syn genomes,mean pro/syn alignments,query_y,mit9313,mit9313 alignments
0,ORF533_BX548175.1:237054:237281,BX548175.1,237054,237281,237054,237281,+,MLYLVAPAGDGRDVYATLYAQRMFFLVTLQPRGAQFEVIPYQDARH...,228,75,...,True,True,other,NaN,ORF533_BX548175.1:237054:237281,52.0,1.019231,ORF533_BX548175.1:237054:237281,1.0,1.0
1,ORF2049_BX548175.1:913137:913382,BX548175.1,913137,913382,913137,913382,+,MSEEQLKAFIAKVQGDTSLQEQLKAEGADVVAIAKAAGFSITTEDL...,246,81,...,True,True,ProcA,1.0,ORF2049_BX548175.1:913137:913382,5.0,12.000000,ORF2049_BX548175.1:913137:913382,1.0,34.0
2,ORF2068_BX548175.1:922530:922661,BX548175.1,922530,922661,922530,922661,+,MMGKTKQHKKLVSWAEKAELCATREEAQKLIRKAEKAHAKLES,132,43,...,True,True,N/A,NaN,ORF2068_BX548175.1:922530:922661,11.0,1.000000,ORF2068_BX548175.1:922530:922661,1.0,1.0
3,ORF2106_BX548175.1:940263:940514,BX548175.1,940263,940514,940263,940514,+,MGFLHELEISDGGHRFELASHDHPDHHHLVCVRCGRTEEFESKPVL...,252,83,...,True,True,other,NaN,ORF2106_BX548175.1:940263:940514,51.0,1.019608,ORF2106_BX548175.1:940263:940514,1.0,1.0
4,ORF2222_BX548175.1:986481:986612,BX548175.1,986481,986612,986481,986612,+,MAETATIRSDPDSSRNNRGKFVVAGYVLELAKIISDRRANQSS,132,43,...,True,True,hypothetical,NaN,ORF2222_BX548175.1:986481:986612,24.0,1.000000,ORF2222_BX548175.1:986481:986612,1.0,1.0


# blast vs pro syn

In [45]:
def prosyn_blast_to_description(blast_rec):
    query_id = blast_rec.query.replace(' unnamed protein product','').replace('lcl|','')
    results = [{
        'query' : query_id,
        'pro_syn_description' : d.title.split('=', maxsplit=1)[-1],
        'pro_syn_e' : d.e,
        'pro_syn_coverage': np.round(np.mean([hsp.align_length / blast_rec.query_length for hsp in a.hsps]), decimals=2),
        'pro_syn_identity': np.round(np.mean([hsp.identities / blast_rec.query_length for hsp in a.hsps]), decimals=2),
        
        }
        for d,a in zip(blast_rec.descriptions, blast_rec.alignments)
    ]
    return results

In [46]:
fh = open(blast_vs_pro_dpath)
blast_records = NCBIXML.parse(fh)
reslist = [prosyn_blast_to_description(b) for b in blast_records]
blast_vs_pro_df = pd.DataFrame([i for lst in reslist for i in lst])

In [47]:
found_sorf_df = pd.merge(found_sorf_df, 
         blast_vs_pro_df.loc[blast_vs_pro_df.groupby('query').pro_syn_e.idxmin()].reset_index(drop=True),
         left_on='orfid', right_on='query', how='left')

In [48]:
found_sorf_df.head()

,orfid,contig,left,right,start,stop,strand,aaseq,len_nn,len_aa,...,pro/syn genomes,mean pro/syn alignments,query_y,mit9313,mit9313 alignments,query,pro_syn_description,pro_syn_e,pro_syn_coverage,pro_syn_identity
0,ORF533_BX548175.1:237054:237281,BX548175.1,237054,237281,237054,237281,+,MLYLVAPAGDGRDVYATLYAQRMFFLVTLQPRGAQFEVIPYQDARH...,228,75,...,52.0,1.019231,ORF533_BX548175.1:237054:237281,1.0,1.0,ORF533_BX548175.1:237054:237281,gb|KZR77542.1| hypothetical protein PMIT1323_0...,1.367810e-51,1.0,1.00
1,ORF2049_BX548175.1:913137:913382,BX548175.1,913137,913382,913137,913382,+,MSEEQLKAFIAKVQGDTSLQEQLKAEGADVVAIAKAAGFSITTEDL...,246,81,...,5.0,12.000000,ORF2049_BX548175.1:913137:913382,1.0,34.0,ORF2049_BX548175.1:913137:913382,ref|WP_011130214.1| Nif11-like leader peptide ...,4.276760e-53,1.0,1.00
2,ORF2068_BX548175.1:922530:922661,BX548175.1,922530,922661,922530,922661,+,MMGKTKQHKKLVSWAEKAELCATREEAQKLIRKAEKAHAKLES,132,43,...,11.0,1.000000,ORF2068_BX548175.1:922530:922661,1.0,1.0,ORF2068_BX548175.1:922530:922661,gb|ABM78103.1| Hypothetical protein P9303_1356...,1.409580e-20,1.0,0.93
3,ORF2106_BX548175.1:940263:940514,BX548175.1,940263,940514,940263,940514,+,MGFLHELEISDGGHRFELASHDHPDHHHLVCVRCGRTEEFESKPVL...,252,83,...,51.0,1.019608,ORF2106_BX548175.1:940263:940514,1.0,1.0,ORF2106_BX548175.1:940263:940514,gb|KZR75617.1| Peroxide-responsive repressor P...,4.417690e-57,1.0,1.00
4,ORF2222_BX548175.1:986481:986612,BX548175.1,986481,986612,986481,986612,+,MAETATIRSDPDSSRNNRGKFVVAGYVLELAKIISDRRANQSS,132,43,...,24.0,1.000000,ORF2222_BX548175.1:986481:986612,1.0,1.0,ORF2222_BX548175.1:986481:986612,gb|KZR72670.1| hypothetical protein PMIT1313_0...,1.382470e-24,1.0,1.00


# blast vs swisprot

In [49]:
def swisprot_blast_to_description(blast_rec):
    query_id = blast_rec.query.replace(' unnamed protein product','').replace('lcl|','')
    results = [{
        'query' : query_id,
        'swis_description' : d.title.split('=', maxsplit=1)[-1],
        'swis_e' : d.e,
        'swis_coverage': np.round(np.mean([hsp.align_length / blast_rec.query_length for hsp in a.hsps]), decimals=2),
        'swis_identity': np.round(np.mean([hsp.identities / blast_rec.query_length for hsp in a.hsps]), decimals=2),
        
        }
        for d,a in zip(blast_rec.descriptions, blast_rec.alignments)
    ]
    return results

In [50]:
fh = open(blast_vs_swisprot_dpath)
blast_records = NCBIXML.parse(fh)
reslist = [swisprot_blast_to_description(b) for b in blast_records]
blast_vs_swis_df = pd.DataFrame([i for lst in reslist for i in lst])

In [51]:
found_sorf_df = pd.merge(found_sorf_df, 
         blast_vs_swis_df.loc[blast_vs_swis_df.groupby('query').swis_e.idxmin()].reset_index(drop=True),
         left_on='orfid', right_on='query', how='left')

In [52]:
found_sorf_df.otype.value_counts()

Annotated       127
Out of Frame     22
Intergenic       11
In Frame          7
Antisense         2
Name: otype, dtype: int64

In [53]:
found_sorf_df.dropna(subset=['swis_description']).otype.value_counts()

Annotated       35
In Frame         3
Out of Frame     3
Name: otype, dtype: int64

In [54]:
found_sorf_df.shape

(169, 94)

# Analysis

In [55]:
[(i,len(j)) for i,j in enumerate(real_clusters, start=1)]

[(1, 19),
 (2, 2),
 (3, 2),
 (4, 2),
 (5, 2),
 (6, 2),
 (7, 2),
 (8, 2),
 (9, 2),
 (10, 3),
 (11, 3)]

In [56]:
found_sorf_df.groupby(['cluster_id','family2'])['product'].value_counts()

cluster_id  family2       product                                                   
1.0         ProcA         Nif11-like leader peptide family natural product precursor    18
                          lantipeptide                                                   1
2.0         N/A                                                                          1
            hypothetical  hypothetical protein                                           1
3.0         HLIP          high light inducible protein                                   2
4.0         hypothetical  hypothetical protein                                           2
5.0         hypothetical  hypothetical protein                                           2
6.0         hypothetical  hypothetical protein                                           2
7.0         hypothetical  hypothetical protein                                           2
8.0         hypothetical  hypothetical protein                                           1
     

In [57]:
found_sorf_df.loc[found_sorf_df.cluster_id.isna() & ~found_sorf_df.family2.isin(['other'])].groupby(['family2'])['product'].value_counts()

family2       product                                      
CCRG-1        guanylate-binding protein                         1
DUF           DUF4278 domain-containing protein                 2
              DUF1816 domain-containing protein                 1
              DUF1995 family protein                            1
              DUF2470 domain-containing protein                 1
              DUF2555 domain-containing protein                 1
              DUF2862 domain-containing protein                 1
              DUF2949 domain-containing protein                 1
              DUF2973 domain-containing protein                 1
              DUF2996 domain-containing protein                 1
              DUF3136 domain-containing protein                 1
              DUF3148 domain-containing protein                 1
              DUF3181 family protein                            1
              DUF3252 domain-containing protein                 1
              DU

In [58]:
found_sorf_df.loc[found_sorf_df.cluster_id.isna() & found_sorf_df.family2.isin(['other'])]['product'].value_counts()

2Fe-2S iron-sulfur cluster binding domain-containing protein                                    2
NAD(P)H-quinone oxidoreductase subunit O                                                        1
chlororespiratory reduction protein 7                                                           1
NAD(P)-dependent oxidoreductase                                                                 1
BolA family transcriptional regulator                                                           1
transcriptional repressor                                                                       1
CopG family transcriptional regulator                                                           1
YlxR family protein                                                                             1
glutaredoxin 3                                                                                  1
16S rRNA (cytidine(1402)-2'-O)-methyltransferase                                                1
4a-hydroxytetrahydro

In [59]:
found_sorf_df.loc[found_sorf_df.family2.isin(['other']), 'product'].value_counts()

2Fe-2S iron-sulfur cluster binding domain-containing protein                                    2
NAD(P)H-quinone oxidoreductase subunit O                                                        1
NifU family protein                                                                             1
ssl1498 family light-harvesting-like protein                                                    1
NAD(P)-dependent oxidoreductase                                                                 1
BolA family transcriptional regulator                                                           1
transcriptional repressor                                                                       1
chemotaxis protein                                                                              1
CopG family transcriptional regulator                                                           1
YlxR family protein                                                                             1
glutaredoxin 3      

In [60]:
pd.set_option('display.max_colwidth', -1)
found_sorf_df.groupby(['cluster_id', 'otype', 'orfid', 'product', #'swis_description', #'family'
                      ])['family2'].unique().str.join(',').reset_index()

,cluster_id,otype,orfid,product,family2
0,1.0,Annotated,ORF15620_BX548175.1:2243295:2243047,Nif11-like leader peptide family natural product precursor,ProcA
1,1.0,Annotated,ORF15637_BX548175.1:2235903:2235652,Nif11-like leader peptide family natural product precursor,ProcA
2,1.0,Annotated,ORF15638_BX548175.1:2235486:2235229,lantipeptide,ProcA
3,1.0,Annotated,ORF15639_BX548175.1:2235150:2234896,Nif11-like leader peptide family natural product precursor,ProcA
4,1.0,Annotated,ORF20045_BX548175.1:274995:274750,Nif11-like leader peptide family natural product precursor,ProcA
5,1.0,Annotated,ORF20053_BX548175.1:273630:273355,Nif11-like leader peptide family natural product precursor,ProcA
6,1.0,Annotated,ORF2049_BX548175.1:913137:913382,Nif11-like leader peptide family natural product precursor,ProcA
7,1.0,Annotated,ORF20960_BX548175.1:2244944:2244663,Nif11-like leader peptide family natural product precursor,ProcA
8,1.0,Annotated,ORF20982_BX548175.1:2236277:2236026,Nif11-like leader peptide family natural product precursor,ProcA
9,1.0,Annotated,ORF25353_BX548175.1:276446:276159,Nif11-like leader peptide family natural product precursor,ProcA


In [61]:
found_sorf_df.loc[found_sorf_df.cluster_id.isna()]['otype'].value_counts()

Annotated       95
Out of Frame    17
Intergenic      10
In Frame        4 
Antisense       2 
Name: otype, dtype: int64

In [62]:
found_sorf_df.pivot_table(columns='otype', index='family2',aggfunc='count', values='orfid').reset_index()

otype,family2,Annotated,Antisense,In Frame,Intergenic,Out of Frame
0,CCRG-1,1.0,NaN,NaN,NaN,NaN
1,CCRG-2,2.0,NaN,NaN,NaN,NaN
2,DUF,12.0,NaN,2.0,NaN,3.0
3,HLIP,3.0,NaN,NaN,NaN,1.0
4,N/A,NaN,NaN,NaN,11.0,NaN
5,ProcA,16.0,NaN,2.0,NaN,1.0
6,carboxysome,2.0,NaN,NaN,NaN,NaN
7,hypothetical,53.0,2.0,1.0,NaN,9.0
8,other,22.0,NaN,1.0,NaN,7.0
9,photosystem,5.0,NaN,1.0,NaN,NaN


In [63]:
found_sorf_df.otype.value_counts()

Annotated       127
Out of Frame    22 
Intergenic      11 
In Frame        7  
Antisense       2  
Name: otype, dtype: int64

In [64]:
found_sorf_df.columns

Index(['orfid', 'contig', 'left', 'right', 'start', 'stop', 'strand', 'aaseq',
       'len_nn', 'len_aa', 'len_aa3', 'rast_left', 'rast_right', 'contig_id',
       'gene_id', 'location', 'type', 'start_r', 'stop_r', 'function',
       'figfam', 'nucleotide_sequence', 'aa_sequence', 'genome', 'left_r',
       'right_r', 'overlap_location', 'overlap_strand', 'overlap_gene_type',
       'overlap_count', 'is_same_strand', 'is_out_of_frame', 'is_inside',
       'is_upstream', 'is_downstream', 'overlap_type', 'otype',
       'merge_location', 'contig_id_overlap', 'gene_id_overlap', 'pmt_id',
       'type_overlap', 'location_overlap', 'strand_overlap', 'start_overlap',
       'stop_overlap', 'left_overlap', 'right_overlap', 'function_overlap',
       'genome_overlap', 'old_locus_tag', 'product', 'db_xref', 'protein_id',
       'figfam_overlap', 'nucleotide_sequence_overlap', 'aa_sequence_overlap',
       'family', 'Gene', 'Accession', 'Description', 'Score', 'Coverage',
       'Proteins', 'Un

In [65]:
pd.set_eng_float_format()

1.35

In [66]:
pd.set_option('display.max_colwidth', -1)
pd.set_option('display.precision', 0)
found_sorf_df.dropna(subset=['swis_description'])[['family2','otype', 'pro/syn genomes',
       'mean pro/syn alignments', 'product', 'swis_description', 'swis_e',
       'swis_coverage', 'swis_identity' ]]

,family2,otype,pro/syn genomes,mean pro/syn alignments,product,swis_description,swis_e,swis_coverage,swis_identity
3,other,Out of Frame,51,1,transcriptional repressor,Ferric uptake regulation protein; Short=Ferric uptake regulator [Synechococcus elongatus PCC 7942 = FACHB-805],2e-14,9e-01,4e-01
8,other,In Frame,49,1,FADH(2)-oxidizing methylenetetrahydrofolate--tRNA-(uracil(54)-C(5))- methyltransferase TrmFO,Methylenetetrahydrofolate--tRNA-(uracil-5-)-methyltransferase TrmFO; AltName: Full=Folate-dependent tRNA (uracil-5-)-methyltransferase; AltName: Full=Folate-dependent tRNA(M-5-U54)-methyltransferase [Prochlorococcus marinus str. MIT 9303],1e-54,1e+00,9e-01
9,other,Annotated,51,1,BolA family transcriptional regulator,Uncharacterized protein ssr3122 [Synechocystis sp. PCC 6803 substr. Kazusa],5e-28,1e+00,6e-01
13,HLIP,Out of Frame,54,2,high light inducible protein,Uncharacterized protein ycf17; AltName: Full=ORF48 [Cyanophora paradoxa],5e-07,6e-01,3e-01
19,ribosomal,Annotated,86,1,30S ribosomal protein S19,30S ribosomal protein S19 [Prochlorococcus marinus str. MIT 9303] >sp|Q7TUP5.1| RecName: Full=30S ribosomal protein S19 [Prochlorococcus marinus str. MIT 9313],9e-63,1e+00,1e+00
20,ribosomal,Annotated,56,1,50S ribosomal protein L29,50S ribosomal protein L29 [Prochlorococcus marinus str. MIT 9313],1e-44,1e+00,1e+00
24,other,Annotated,52,3,response regulator transcription factor,Oxygen regulatory protein NreC; AltName: Full=Nitrogen regulation protein C [Staphylococcus aureus RF122] >sp|Q6GE42.1| RecName: Full=Oxygen regulatory protein NreC; AltName: Full=Nitrogen regulation protein C [Staphylococcus aureus subsp. aureus MRSA252],1e-10,6e-01,3e-01
36,ribosomal,Annotated,83,1,30S ribosomal protein S15,30S ribosomal protein S15 [Prochlorococcus marinus str. MIT 9303] >sp|Q7TV07.1| RecName: Full=30S ribosomal protein S15 [Prochlorococcus marinus str. MIT 9313],1e-59,1e+00,1e+00
37,ribosomal,Annotated,80,1,30S ribosomal protein S18,30S ribosomal protein S18 [Prochlorococcus marinus str. MIT 9303] >sp|Q7TUZ8.1| RecName: Full=30S ribosomal protein S18 [Prochlorococcus marinus str. MIT 9313],1e-46,1e+00,1e+00
38,ribosomal,Annotated,78,1,50S ribosomal protein L32,50S ribosomal protein L32 [Prochlorococcus marinus str. MIT 9303] >sp|Q7V7I8.1| RecName: Full=50S ribosomal protein L32 [Prochlorococcus marinus str. MIT 9313],9e-37,1e+00,1e+00


In [71]:
pd.set_option('display.max_colwidth', -1)
pd.set_option('display.precision', 0)
found_sorf_df.dropna(subset=['swis_description']).loc[found_sorf_df.otype == 'Out of Frame',
      ['family2','otype', 'pro/syn genomes',
       'mean pro/syn alignments', 'product', 'swis_description', 'swis_e',
       'swis_coverage', 'swis_identity' ]]

,family2,otype,pro/syn genomes,mean pro/syn alignments,product,swis_description,swis_e,swis_coverage,swis_identity
3,other,Out of Frame,51,1,transcriptional repressor,Ferric uptake regulation protein; Short=Ferric uptake regulator [Synechococcus elongatus PCC 7942 = FACHB-805],2e-14,9e-01,4e-01
13,HLIP,Out of Frame,54,2,high light inducible protein,Uncharacterized protein ycf17; AltName: Full=ORF48 [Cyanophora paradoxa],5e-07,6e-01,3e-01
55,ribosomal,Out of Frame,66,1,30S ribosomal protein S13,30S ribosomal protein S13 [Synechococcus sp. CC9311],8e-26,1e+00,1e+00


In [72]:
pd.set_option('display.max_colwidth', 100)
pd.set_option('display.precision', 0)
found_sorf_df.loc[found_sorf_df.otype.isin(['Intergenic']),
                  ['orfid', 'mit9313 alignments','pro/syn genomes',
                   'mean pro/syn alignments', #'swis_description', 
                   'pro_syn_description','pro_syn_e', 'pro_syn_coverage',
       'pro_syn_identity', 'cluster_id']]

,orfid,mit9313 alignments,pro/syn genomes,mean pro/syn alignments,pro_syn_description,pro_syn_e,pro_syn_coverage,pro_syn_identity,cluster_id
2,ORF2068_BX548175.1:922530:922661,1,11,1,gb|ABM78103.1| Hypothetical protein P9303_13561 [Prochlorococcus marinus str. MIT 9303],1e-20,1e+00,9e-01,NaN
5,ORF2321_BX548175.1:1030986:1031147,3,1,3,gb|KZR65522.1| hypothetical protein PMIT1306_00434 [Prochlorococcus sp. MIT 1306] >gb|KZR65553.1...,5e-32,1e+00,1e+00,2
46,ORF7466_BX548175.1:1097200:1097379,1,28,1,ref|WP_011825714.1| unnamed protein product >gb|ABM77811.1| Conserved hypothetical protein [Proc...,2e-39,1e+00,1e+00,NaN
76,ORF13811_BX548175.1:1709954:1710130,1,33,1,ref|WP_011824978.1| hypothetical protein [Prochlorococcus marinus] >gb|ABM77051.1| membrane prot...,2e-33,1e+00,1e+00,NaN
90,ORF16879_BX548175.1:1658121:1657948,1,2,1,emb|CAX32265.1| Conserved hypothetical protein [Prochlorococcus marinus str. MIT 9313],4e-36,1e+00,1e+00,NaN
100,ORF18609_BX548175.1:926268:926035,1,5,1,gb|KGG27689.1| hypothetical protein EV12_1119 [Prochlorococcus sp. MIT 0701] >gb|KGG31928.1| hyp...,5e-26,8e-01,6e-01,NaN
108,ORF19962_BX548175.1:317208:317134,1,1,1,emb|CAE20454.1| hypothetical [Prochlorococcus marinus str. MIT 9313],2e-10,1e+00,1e+00,NaN
122,ORF22713_BX548175.1:1476551:1476351,1,16,1,emb|CAX32212.1| Conserved hypothetical protein [Prochlorococcus marinus str. MIT 9313],8e-43,1e+00,1e+00,NaN
161,ORF30023_BX548175.1:644083:643928,1,32,1,ref|WP_011129965.1| MULTISPECIES: hypothetical protein [Synechococcales] >gb|ABM78406.1| conserv...,1e-28,1e+00,1e+00,NaN
164,ORF30529_BX548175.1:408367:408233,1,19,1,ref|WP_011129739.1| MULTISPECIES: hypothetical protein [Prochlorococcus] >gb|NMO84559.1| hypothe...,3e-26,1e+00,1e+00,NaN


In [68]:
found_sorf_df.orfid.value_counts()

ORF2321_BX548175.1:1030986:1031147     1
ORF5528_BX548175.1:226567:226794       1
ORF19358_BX548175.1:599313:599146      1
ORF7466_BX548175.1:1097200:1097379     1
ORF29905_BX548175.1:697087:696860      1
                                      ..
ORF21363_BX548175.1:2060999:2060784    1
ORF2068_BX548175.1:922530:922661       1
ORF25338_BX548175.1:284903:284670      1
ORF30630_BX548175.1:358513:358319      1
ORF14108_BX548175.1:1840259:1840525    1
Name: orfid, Length: 169, dtype: int64

In [69]:
found_sorf_df.head()

,orfid,contig,left,right,start,stop,strand,aaseq,len_nn,len_aa,...,query_x,pro_syn_description,pro_syn_e,pro_syn_coverage,pro_syn_identity,query_y,swis_description,swis_e,swis_coverage,swis_identity
0,ORF533_BX548175.1:237054:237281,BX548175.1,237054,237281,237054,237281,+,MLYLVAPAGDGRDVYATLYAQRMFFLVTLQPRGAQFEVIPYQDARHYAELHLTHCRRDRSPEYESWQQLFAQTFI,228,75,...,ORF533_BX548175.1:237054:237281,gb|KZR77542.1| hypothetical protein PMIT1323_00917 [Prochlorococcus marinus str. MIT 1323],1e-51,1,1e+00,NaN,NaN,NaN,NaN,NaN
1,ORF2049_BX548175.1:913137:913382,BX548175.1,913137,913382,913137,913382,+,MSEEQLKAFIAKVQGDTSLQEQLKAEGADVVAIAKAAGFSITTEDLNTHRQTLSDRELEGVAGGTTAFTGVDTESIAFCCS,246,81,...,ORF2049_BX548175.1:913137:913382,ref|WP_011130214.1| Nif11-like leader peptide family natural product precursor [Prochlorococcus marinus] >emb|CAE21011.1| hypothetical [Prochloroc...,4e-53,1,1e+00,NaN,NaN,NaN,NaN,NaN
2,ORF2068_BX548175.1:922530:922661,BX548175.1,922530,922661,922530,922661,+,MMGKTKQHKKLVSWAEKAELCATREEAQKLIRKAEKAHAKLES,132,43,...,ORF2068_BX548175.1:922530:922661,gb|ABM78103.1| Hypothetical protein P9303_13561 [Prochlorococcus marinus str. MIT 9303],1e-20,1,9e-01,NaN,NaN,NaN,NaN,NaN
3,ORF2106_BX548175.1:940263:940514,BX548175.1,940263,940514,940263,940514,+,MGFLHELEISDGGHRFELASHDHPDHHHLVCVRCGRTEEFESKPVLEAGRSAAERIGFQLIESTLNVRALCPSCQRPSTTNVH,252,83,...,ORF2106_BX548175.1:940263:940514,gb|KZR75617.1| Peroxide-responsive repressor PerR [Prochlorococcus marinus str. MIT 1323],4e-57,1,1e+00,ORF2106_BX548175.1:940263:940514,Ferric uptake regulation protein; Short=Ferric uptake regulator [Synechococcus elongatus PCC 7942 = FACHB-805],2e-14,9e-01,4e-01
4,ORF2222_BX548175.1:986481:986612,BX548175.1,986481,986612,986481,986612,+,MAETATIRSDPDSSRNNRGKFVVAGYVLELAKIISDRRANQSS,132,43,...,ORF2222_BX548175.1:986481:986612,gb|KZR72670.1| hypothetical protein PMIT1313_00025 [Prochlorococcus marinus str. MIT 1313] >gb|KZR75200.1| hypothetical protein PMIT1318_00285 [Pr...,1e-24,1,1e+00,NaN,NaN,NaN,NaN,NaN


In [76]:
blastn_vs_pro_df.loc[blastn_vs_pro_df['query'].str.contains('ORF2321_BX548175.1:1030986:1031147')]

,query,pro_syn,pro_syn_hsps,pro_syn_coverage,pro_syn_identity
148,ORF2321_BX548175.1:1030986:1031147,Prochlorococcus marinus MIT9313 complete genome,3,1e+00,8e-01
149,ORF2321_BX548175.1:1030986:1031147,"Prochlorococcus marinus str. MIT 9303, complete genome",3,1e+00,8e-01


In [81]:
found_sorf_df.nlargest(10, ['pro/syn genomes'])

,orfid,contig,left,right,start,stop,strand,aaseq,len_nn,len_aa,...,query_x,pro_syn_description,pro_syn_e,pro_syn_coverage,pro_syn_identity,query_y,swis_description,swis_e,swis_coverage,swis_identity
19,ORF3964_BX548175.1:1838136:1838411,BX548175.1,1838136,1838411,1838136,1838411,+,MGRSLKKGPFIADSLLRKLEKQNADDDKSVIKTWSRASTILPMMIGHTIAVHNGRSHVPVFITEQMVGHKLGEFAPTRTFKGHIKDKKGGR,276,91,...,ORF3964_BX548175.1:1838136:1838411,30S ribosomal protein S19 [Prochlorococcus marinus str. MIT 9303] >sp|Q7TUP5.1| RecName: Full=30...,6e-63,1,1,ORF3964_BX548175.1:1838136:1838411,30S ribosomal protein S19 [Prochlorococcus marinus str. MIT 9303] >sp|Q7TUP5.1| RecName: Full=30...,9e-63,1,1
74,ORF13351_BX548175.1:1509413:1509679,BX548175.1,1509413,1509679,1509413,1509679,+,MAHKKGTGSTRNGRDSNSKRLGVKAYGGETVTAGSILIRQRGTSVLPGVNVGQGKDDTLFALTDGVVAFESIRRSLRNRKRISVVASS,267,88,...,ORF13351_BX548175.1:1509413:1509679,50S ribosomal protein L27 [Prochlorococcus marinus str. MIT 9313] >emb|CAE21596.1| 50S ribosomal...,2e-57,1,1,ORF13351_BX548175.1:1509413:1509679,50S ribosomal protein L27 [Prochlorococcus marinus str. MIT 9313],3e-57,1,1
80,ORF14108_BX548175.1:1840259:1840525,BX548175.1,1840259,1840525,1840259,1840525,+,MALKERLGTVVSDKMDKTVVVAVENRFPHPIYQKTVSRTTRYKAHDAGNTCRVGDRVRITETRPLSRSKRWTVAEVLSHSPKAQEVST,267,88,...,ORF14108_BX548175.1:1840259:1840525,30S ribosomal protein S17 [Prochlorococcus marinus str. MIT 9313] >gb|KZR68352.1| 30S ribosomal ...,2e-60,1,1,ORF14108_BX548175.1:1840259:1840525,30S ribosomal protein S17 [Prochlorococcus marinus str. MIT 9313],3e-60,1,1
81,ORF14128_BX548175.1:1850909:1851190,BX548175.1,1850909,1851190,1850909,1851190,+,MPKPDIHPNWYPDAKVICNGEVVMTTGSTQPELHVDVWSGNHPFFTGTQKILDTEGRVDRFMRKYGMGSADAAADEKKPDAKNNNKDNTSKED,282,93,...,ORF14128_BX548175.1:1850909:1851190,50S ribosomal protein L31 [Prochlorococcus marinus str. MIT 9313] >gb|KZR73915.1| 50S ribosomal ...,2e-66,1,1,ORF14128_BX548175.1:1850909:1851190,50S ribosomal protein L31 [Prochlorococcus marinus str. MIT 9313],2e-66,1,1
36,ORF6596_BX548175.1:698671:698940,BX548175.1,698671,698940,698671,698940,+,MSLDTTEKQQLINANQTHGTDTGSVEVQVAMLSERITKLSSHLQENKHDFSSRQGLLKMIGRRKRLLSYVRGKSEQRYNGLITKLGIRG,270,89,...,ORF6596_BX548175.1:698671:698940,30S ribosomal protein S15 [Prochlorococcus marinus str. MIT 9303] >sp|Q7TV07.1| RecName: Full=30...,1e-59,1,1,ORF6596_BX548175.1:698671:698940,30S ribosomal protein S15 [Prochlorococcus marinus str. MIT 9303] >sp|Q7TV07.1| RecName: Full=30...,1e-59,1,1
104,ORF19047_BX548175.1:750573:750337,BX548175.1,750337,750573,750573,750337,-,MSRVCQLTGTRANNGMAVSHSHIRTKKLQQANLQQRRLWWAEGNRWLKLRVSTRALKTIQKKGLGVYAKSLGIDLNKI,237,78,...,ORF19047_BX548175.1:750573:750337,50S ribosomal protein L28 [Prochlorococcus marinus str. MIT 9303] >sp|Q7V7P3.1| RecName: Full=50...,3e-51,1,1,ORF19047_BX548175.1:750573:750337,50S ribosomal protein L28 [Prochlorococcus marinus str. MIT 9303] >sp|Q7V7P3.1| RecName: Full=50...,4e-51,1,1
37,ORF6827_BX548175.1:807277:807498,BX548175.1,807277,807498,807277,807498,+,MSSSFFKKRLSPIKPGDPIDYKDVDLLKKFITDRGKILPRRLTGLTSKQQRDLTNAVKRARIIALLPFVNPEG,222,73,...,ORF6827_BX548175.1:807277:807498,30S ribosomal protein S18 [Prochlorococcus marinus str. MIT 9303] >sp|Q7TUZ8.1| RecName: Full=30...,9e-47,1,1,ORF6827_BX548175.1:807277:807498,30S ribosomal protein S18 [Prochlorococcus marinus str. MIT 9303] >sp|Q7TUZ8.1| RecName: Full=30...,1e-46,1,1
66,ORF11919_BX548175.1:807044:807238,BX548175.1,807044,807238,807044,807238,+,MAKNKGVRIVITLECNECRSNPAKRSPGVSRYTTEKNRRNTTERLEIKKFCPHCNKSTPHKEIK,195,64,...,ORF11919_BX548175.1:807044:807238,50S ribosomal protein L33 [Prochlorococcus marinus str. MIT 9313] >emb|CAE20914.1| 50S ribosomal...,5e-41,1,1,ORF11919_BX548175.1:807044:807238,50S ribosomal protein L33 [Prochlorococcus marinus str. MIT 9313],7e-41,1,1
97,ORF18186_BX548175.1:1098174:1097998,BX548175.1,1097998,1098174,1098174,1097998,-,MAQVTVGENEGVESALRRFKRAVSKAGIFSDLKRIRHHETPVEKYKRKAQQRRRSRRR,177,58,...,ORF18186_BX548175.1:1098174:1097998,30S ribosomal protein S21 [

In [82]:
found_sorf_df.nsmallest(10, ['pro/syn genomes'])

,orfid,contig,left,right,start,stop,strand,aaseq,len_nn,len_aa,...,query_x,pro_syn_description,pro_syn_e,pro_syn_coverage,pro_syn_identity,query_y,swis_description,swis_e,swis_coverage,swis_identity
5,ORF2321_BX548175.1:1030986:1031147,BX548175.1,1030986,1031147,1030986,1031147,+,MLSKSITGMFGRRNKKDAPKEEPTETDLERGYRLAAAKQKEILKELLAQKPRA,162,53,...,ORF2321_BX548175.1:1030986:1031147,gb|KZR65522.1| hypothetical protein PMIT1306_00434 [Prochlorococcus sp. MIT 1306] >gb|KZR65553.1...,5e-32,1e+00,1e+00,NaN,NaN,NaN,NaN,NaN
7,ORF2340_BX548175.1:1038120:1038374,BX548175.1,1038120,1038374,1038120,1038374,+,MYSLFDSVFDVPFGYSIPRDRVVVIPDSQYNKLRAQENERQVAKLEARKEHHSQVIERLNEQISELQAALPAAEPDKELAATKE,255,84,...,ORF2340_BX548175.1:1038120:1038374,ref|WP_011130332.1| guanylate-binding protein [Prochlorococcus marinus] >gb|KZR68145.1| hypothet...,1e-56,1e+00,1e+00,NaN,NaN,NaN,NaN,NaN
14,ORF3835_BX548175.1:1762950:1763246,BX548175.1,1762950,1763246,1762950,1763246,+,MRFIEFLDQRFCNSQEIREINHRGISAGFDGFREVWELDYLYEHFEADIHEIIEETGYLDSDLKDEFDNEETLRMKLVWFAVSTYCLSRTDALVEAKD,297,98,...,ORF3835_BX548175.1:1762950:1763246,ref|WP_011131024.1| hypothetical protein [Prochlorococcus marinus] >gb|KZR66653.1| hypothetical ...,5e-68,1e+00,1e+00,NaN,NaN,NaN,NaN,NaN
25,ORF4637_BX548175.1:2186205:2186360,BX548175.1,2186205,2186360,2186205,2186360,+,MSSSEKELDLRIDHLQKELALAEQERDILRAAKMQQKILMKLLKENPSRQV,156,51,...,ORF4637_BX548175.1:2186205:2186360,ref|WP_155728142.1| MULTISPECIES: hypothetical protein [Prochlorococcus] >gb|KZR61380.1| hypothe...,1e-28,1e+00,1e+00,NaN,NaN,NaN,NaN,NaN
31,ORF5755_BX548175.1:320164:320430,BX548175.1,320164,320430,320164,320430,+,MIMEDPESNASSSNDQTEAFEPPKNRFGIPYLLATDKEEKDMPKKEVLNVLGLQLSAPAGMQNPLLIFFGLIGVNIAIIGYFTYMLNK,267,88,...,ORF5755_BX548175.1:320164:320430,ref|WP_063403417.1| unnamed protein product >gb|KZR70057.1| hypothetical protein PMIT1313_00703 ...,1e-58,1e+00,1e+00,NaN,NaN,NaN,NaN,NaN
41,ORF7084_BX548175.1:929743:929898,BX548175.1,929743,929898,929743,929898,+,MRQSFAGTTGAVQPLATPRIQSIREKISIQELELRLTRLGLATTTHTPNPS,156,51,...,ORF7084_BX548175.1:929743:929898,ref|WP_155724952.1| MULTISPECIES: hypothetical protein [Prochlorococcus] >gb|KZR64439.1| hypothe...,4e-30,1e+00,1e+00,NaN,NaN,NaN,NaN,NaN
42,ORF7163_BX548175.1:972145:972303,BX548175.1,972145,972303,972145,972303,+,MPSVKPRVGWIIDPELLSLLKELADKSGRTVPKEAEQAIKNWLQLHGNLPQD,159,52,...,ORF7163_BX548175.1:972145:972303,ref|WP_157859818.1| hypothetical protein [Prochlorococcus marinus] >emb|CAX31986.1| Hypothetical...,8e-32,1e+00,1e+00,NaN,NaN,NaN,NaN,NaN
43,ORF7228_BX548175.1:995548:995733,BX548175.1,995548,995733,995548,995733,+,MNNTIKRTSVTRGLTNIHTIKLAVLDAIEYAEQRGDLNLALAYAALLKKDLDPLIKERLGL,186,61,...,ORF7228_BX548175.1:995548:995733,ref|WP_041384390.1| hypothetical protein [Prochlorococcus marinus] >emb|CAX32005.1| Conserved hy...,2e-37,1e+00,1e+00,NaN,NaN,NaN,NaN,NaN
44,ORF7288_BX548175.1:1022485:1022760,BX548175.1,1022485,1022760,1022485,1022760,+,MTMNQTYFDELFERWQAADEVDDETLQLIKDTVRFTQETSDNSLRGHARQLLSDVYHLPSTPQEIALIGEALEGTGTVPPEIPPEDNGYGL,276,91,...,ORF7288_BX548175.1:1022485:1022760,ref|WP_011130317.1| hypothetical protein [Prochlorococcus marinus] >emb|CAE21116.1| hypothetical...,9e-62,1e+00,1e+00,NaN,NaN,NaN,NaN,NaN
45,ORF7312_BX548175.1:1031485:1031757,BX548175.1,1031485,1031757,1031485,1031757,+,MTMNETDFAELFERWQAAEQVDDELIQLIKETVRFSEETGDDTLRDRACDLLSGVFHLPSTPQEIALVGDALEGSGTIPPPIPPEDDEAR,273,90,...,ORF7312_BX548175.1:1031485:1031757,emb|CAE21124.1| hypothetical [Prochlorococcus marinus str. MIT 9313],6e-61,1e+00,1e+00,NaN,NaN,NaN,NaN,NaN
